In [181]:
import pandas as pd
course_data = pd.read_csv('training_pd_new.csv')
user_input = ''

In [182]:
import numpy as np
np.random.seed(42)
to_changed_label = np.random.randint(low=0,high=569,size=50)
course_data.loc[to_changed_label, 'label'] = course_data.loc[to_changed_label, 'label'].apply(lambda x: 1 if x==0 else 0)

In [183]:
course_data['Course_Name'].value_counts()[:10]

Introduction to business analytics                                 15
Senior seminar in economics and finance                            14
Behavioural and sociological finance                               12
Selections and training                                            12
Impact lab                                                          9
Investments and portfolio analysis                                  7
Current issues in asset management and private banking industr      7
Supply chain management                                             7
Investment consulting, institutional business and family office     7
Project management                                                  7
Name: Course_Name, dtype: int64

In [184]:
train['text']

['Insurance ManagerThe political economy of law and public policyunderstanding of political economy of law and public policy making, knowledge of economic concepts and tools for policy analysis, familiarity with regulation, law and economics, law enforcement, legal systems, public bureaucracy, monopolies, public goods, and rent seeking.',
 'Fraud Operations SpecialistSocial media marketingunderstanding social media marketing principles, applications, and strategies, social media and network theory, popular social media strategies, data analytics tools, analyzing, building, and managing real-world social media marketing campaigns.',
 'Bank TellerAnalysis of economic dataStudying measurement and interpretation of economic variables, modeling their relationships using empirical methods, interpreting headline statistics, describing economic aggregates, modeling economic relationships, and drawing conclusions from observations. ',
 'Insurance ManagerIntermediate financial accounting IIExami

In [185]:
course_data['label'] = course_data['label'].astype('int')

In [186]:
course_data['full_text'] = course_data['Job']+course_data['Course_Name']+course_data['Course_Hard_Skills']

In [187]:
course_data.head()

,Job,Softskills,Hardskills,Course_Name,Course_Description,Course_Hard_Skills,Course_Soft_Skills,label,full_text
0,Mortgage Advisor,"Communication skills, active listening, proble...","Good communication and listening skills, The a...",Investments and portfolio analysis,introduces students to the fundamental princip...,Familiarity with fundamental principles of inv...,Sophisticated assessment of current issues and...,1,Mortgage AdvisorInvestments and portfolio anal...
1,Mortgage Advisor,"Communication skills, active listening, proble...","Good communication and listening skills, The a...",Impact lab,an opportunity to work directly for social ven...,ability to manage tasks and solve real-life pr...,working directly for social ventures under gui...,1,Mortgage AdvisorImpact labability to manage ta...
2,Mortgage Advisor,"Communication skills, active listening, proble...","Good communication and listening skills, The a...",Senior seminar in economics and finance,applies core theoretical knowledge in economic...,"research skills, theoretical knowledge in econ...",NaN,1,Mortgage AdvisorSenior seminar in economics an...
3,Mortgage Advisor,"Communication skills, active listening, proble...","Good communication and listening skills, The a...",Behavioural and sociological finance,covers current developments in finance. Possib...,"Understanding current developments in finance,...",NaN,1,Mortgage AdvisorBehavioural and sociological f...
4,Mortgage Advisor,"Communication skills, active listening, proble...","Good communication and listening skills, The a...",Theory of international trade,deals with economic issues which are related t...,knowledge of economic theory and analytical sk...,NaN,1,Mortgage AdvisorTheory of international tradek...


In [188]:
course_data.shape[0]

570

In [189]:
from transformers import DistilBertTokenizerFast
from collections import defaultdict
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
dataset = defaultdict(dict)
train = dict()
val = dict()
# course_data = shuffle(course_data)
# train['text'] = list(course_data['full_text'][:400])
# val['text'] = list(course_data['full_text'][400:])
# train['label'] = list(course_data['label'][:400])
# val['label'] = list(course_data['label'][400:])
# dataset['train'] = train
# dataset['val'] = val
X_train, X_test, y_train, y_test = train_test_split(list(course_data['full_text']),list(course_data['label']),test_size = 0.3,stratify=course_data['label'],shuffle=True,random_state=42)
train['text'] = X_train
train['label'] = y_train
val['text'] = X_test
val['label'] = y_test
dataset['train'] = train
dataset['val'] = val

In [190]:
train['text']

['Equity Research AssociateSocial media marketingunderstanding social media marketing principles, applications, and strategies, social media and network theory, popular social media strategies, data analytics tools, analyzing, building, and managing real-world social media marketing campaigns.',
 'Bank ManagerAdvanced database managementadvanced database concepts such as views, triggers, stored procedures, SQL*Plus, database administration, and performance tuning, data mining techniques including decision trees, neural networks, and clustering, along with their applications in business.',
 'Accounts ManagerSocial media marketingunderstanding social media marketing principles, applications, and strategies, social media and network theory, popular social media strategies, data analytics tools, analyzing, building, and managing real-world social media marketing campaigns.',
 'Retirement Planning SpecialistDecision and risk analysis IIadvanced quantitative modeling, discrete and continuous

In [191]:
import torch
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(dataset['train']['text'], truncation=True, padding=True)
val_encodings = tokenizer(dataset['val']['text'], truncation=True, padding=True)
class tokenized_dataset(torch.utils.data.Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
    def __getitem__(self, idx):
        item = {'input_ids': torch.tensor(self.data['input_ids'][idx]),
                'attention_mask': torch.tensor(self.data['attention_mask'][idx])}
        item['label'] = torch.tensor(self.label[idx])
        return item
    def __len__(self):
        return len(self.label)
    
small_tokenized_dataset = defaultdict()
small_tokenized_dataset['train']=tokenized_dataset(train_encodings,list(dataset['train']['label']))
small_tokenized_dataset['val'] =tokenized_dataset(val_encodings,list(dataset['val']['label']))

small_tokenized_dataset['train'][:2]

{'input_ids': tensor([[  101, 10067,  2470,  9228, 10085,  4818,  2865,  5821, 20824, 24911,
           2591,  2865,  5821,  6481,  1010,  5097,  1010,  1998,  9942,  1010,
           2591,  2865,  1998,  2897,  3399,  1010,  2759,  2591,  2865,  9942,
           1010,  2951, 25095,  5906,  1010, 20253,  1010,  2311,  1010,  1998,
           6605,  2613,  1011,  2088,  2591,  2865,  5821,  8008,  1012,   102,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0],
         [  101,  2924,  3208,  4215, 21789,  2094,  7809,  2968,  4215, 21789,
           2094,  7809,  8474,  2107,  2004,  5328,  1010, 27099,  1010,  8250,
         

In [192]:
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm
import os 
from transformers import DistilBertForSequenceClassification
import torch
from torch.utils.data import DataLoader

torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
# 设置随机数种子
# ------------------------------------------------------------------------------------------------------------------------------
# Write your code here
import datetime
train_data_loader = DataLoader(small_tokenized_dataset['train'],batch_size=12,shuffle=True)
val_data_loader = DataLoader(small_tokenized_dataset['val'],batch_size=12)
# Define your model. optimizer, hyper-parameter and etc.
from sklearn.metrics import accuracy_score
distil_bert_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=1,problem_type="binary_label_classification")
optimizer = AdamW(distil_bert_model.parameters(),betas=(0.9,0.999),lr=5e-5,eps=1e-5,weight_decay=1e-3)
num_epochs = 8
distil_bert_model.cuda()
bceloss = torch.nn.BCELoss()
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=3, num_training_steps=num_epochs)
start_time = datetime.datetime.now()
for epoch in range(num_epochs):
    #train and evaluate your model
    # training
    distil_bert_model.train()
    train_y_pred = []
    train_y_true = []
    for step, batch in enumerate(train_data_loader):
        input_ids = batch['input_ids'].cuda()
        attention_mask = batch['attention_mask'].cuda()
        labels = batch['label'].cuda()
        y_pred = distil_bert_model(input_ids,attention_mask=attention_mask, labels=labels)
        y_pred = torch.sigmoid(y_pred.logits).squeeze().float()
        loss = bceloss(y_pred, labels.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_y_pred.extend((y_pred>0.5).tolist())
        train_y_true.extend(batch['label'].tolist())
    train_acc = accuracy_score(train_y_pred, train_y_true)
    # evaluation
    distil_bert_model.eval()
    val_y_pred = []
    val_y_true = []
    for step, batch in enumerate(val_data_loader):
        input_ids = batch['input_ids'].cuda()
        attention_mask = batch['attention_mask'].cuda()
        labels = batch['label'].cuda()
        y_pred = distil_bert_model(input_ids,attention_mask=attention_mask, labels=labels)
        y_pred = torch.sigmoid(y_pred.logits).squeeze().float()
        val_y_pred.extend((y_pred>0.5).tolist())
        val_y_true.extend(batch['label'].tolist())
    validation_acc = accuracy_score(val_y_pred, val_y_true)
    scheduler.step()
    # print the training process
    print("Epoch {}: train acc = {:.4f}, validation acc = {:.4f}".format(epoch + 1, train_acc, validation_acc))
end_time = datetime.datetime.now()
print('distilbert training time used:',(end_time-start_time).seconds)
torch.save(distil_bert_model.state_dict(),'pytorch_model.pth')
# ------------------------------------------------------------------------------------------------------------------------------

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

Epoch 1: train acc = 0.5764, validation acc = 0.6374
Epoch 2: train acc = 0.6366, validation acc = 0.6433
Epoch 3: train acc = 0.6892, validation acc = 0.7544
Epoch 4: train acc = 0.7820, validation acc = 0.7719
Epoch 5: train acc = 0.8471, validation acc = 0.7602
Epoch 6: train acc = 0.8371, validation acc = 0.7485
Epoch 7: train acc = 0.8847, validation acc = 0.7310
Epoch 8: train acc = 0.9048, validation acc = 0.7368
distilbert training time used: 33


In [193]:
# inference

In [194]:
courses = pd.read_csv('output_cleaned.csv')
course_skill_dict = courses[['Course_Name','Course_Hard_Skills']].set_index('Course_Name').to_dict()

In [197]:
inference = []
user_input = 'finanical boos'
courses.dropna(subset=['Course_Hard_Skills'],inplace=True,axis=0)
courses = courses.reset_index(drop=True)
for i in range(courses.shape[0]):
    
    input_joined = user_input+' '.join(list(courses.loc[i,['Course_Name','Course_Hard_Skills']]))
    #print(input_joined)
    inference.append(input_joined)
#4
import numpy as np
inference = tokenizer(inference,padding=True,truncation=True)
inference = tokenized_dataset(inference,label=np.ones(len(inference['input_ids'])))
infer_loader = DataLoader(inference,batch_size=24,shuffle=False)

In [198]:
inf_y_pred = []
distil_bert_model.eval()
with torch.no_grad():
    for step, batch in enumerate(infer_loader):
            input_ids = batch['input_ids'].cuda()
            attention_mask = batch['attention_mask'].cuda()
            y_pred = distil_bert_model(input_ids,attention_mask=attention_mask)
            inf_y_pred.extend(torch.sigmoid(y_pred.logits[:,0]).tolist())
_,index = torch.topk(torch.tensor(inf_y_pred),10)
course_name = courses.loc[index.tolist(),'Course_Name'].values
skills = []
for c in course_name:
    skills.append(course_skill_dict['Course_Hard_Skills'][c])
print('skill learned:',skills[0])
print()
print('courses recommended:')
for i,c in enumerate(course_name):
    print(i,',',c)

skill learned: knowledge of the practical aspects of China business environment, understanding of important industries and sectors in Shanghai, ability to analyze topics/issues of recent concern, strategic thinking via firm visits, acquisition of in-depth knowledge of learned topics.

courses recommended:
0 , Field experience in China’s business environment
1 , Current issues in asset management and private banking industr
2 , Introduction to business analytics
3 , Impact lab
4 , Investments and portfolio analysis
5 , Reading course
6 , Valuation using financial statements
7 , Creativity, innovation and entrepreneurship in China I
8 , Behavioural and sociological finance
9 , Managing money in asset management and private banking
